# Gradient Descent for Linear Regression (การไล่ระดับสำหรับการถดถอยเชิงเส้น) (LAB ไม่บังคับ)

<figure>
    <center> <img src="./images/C1_W1_L4_S1_Lecture_GD.png"  style="width:800px;height:200px;" ></center>
</figure>

## เป้าหมาย
ในแล็บนี้ คุณจะ:
- อัตโนมัติการปรับปรุงค่า w และ b โดยใช้วิธีการไล่ระดับ (gradient descent)

## เครื่องมือ (Tools)
ในแล็บนี้ เราจะใช้เครื่องมือต่างๆ ดังนี้:
- NumPy: ไลบรารียอดนิยมสำหรับการคำนวณทางวิทยาศาสตร์
- Matplotlib: ไลบรารียอดนิยมสำหรับการพล็อตข้อมูล
- ฟังก์ชันการพล็อตข้อมูลในไฟล์ lab_utils.py: ซึ่งเป็นไฟล์ที่อยู่ในไดเรกทอรี่ปัจจุบัน

In [ ]:
import math, copy
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('./deeplearning.mplstyle')
from lab_utils_uni import plt_house_x, plt_contour_wgrad, plt_divergence, plt_gradients

<a name="toc_40291_2"></a>
## ปัญหา
ให้ใช้ข้อมูลสองชุดเดียวกันกับก่อนหน้านี้  ซึ่งเป็นข้อมูลเกี่ยวกับบ้านสองหลัง:

- บ้านหลังแรก มีขนาด 1,000 ตารางฟุต ขายได้ในราคา $300,000
- บ้านหลังที่สอง มีขนาด 2,000 ตารางฟุต ขายได้ในราคา $500,000

| Size (1000 sqft)     | Price (1000s of dollars) |
| ----------------| ------------------------ |
| 1               | 300                      |
| 2               | 500                      |


In [ ]:
# Load our data set
x_train = np.array([1.0, 2.0])   #features
y_train = np.array([300.0, 500.0])   #target value

<a name="toc_40291_2.0.1"></a>
### Compute_Cost
ฟังก์ชันนี้ถูกพัฒนาในห้องปฏิบัติการที่แล้ว เราจะต้องใช้มันอีกครั้งที่นี่

In [ ]:
#Function to calculate the cost
def compute_cost(x, y, w, b):
   
    m = x.shape[0] 
    cost = 0
    
    for i in range(m):
        f_wb = w * x[i] + b
        cost = cost + (f_wb - y[i])**2
    total_cost = 1 / (2 * m) * cost

    return total_cost

<a name="toc_40291_2.1"></a>
## สรุปการไล่ระดับ (Gradient descent)
จากบทเรียนที่ผ่านมา คุณได้พัฒนาโมเดลเชิงเส้นเพื่อทำนายค่า  $f_{w,b}(x^{(i)})$:
$$f_{w,b}(x^{(i)}) = wx^{(i)} + b \tag{1}$$
ใน regression เชิงเส้น คุณใช้ข้อมูลการฝึก (training data) เพื่อปรับพารามิเตอร์ $w$,$b$ โดยการลดค่าความผิดพลาดระหว่างค่าที่ทำนายได้ $f_{w,b}(x^{(i)})$ กับค่าจริง $y^{(i)}$. ตัวชี้วัดความผิดพลาดนี้ เรียกว่า cost หรือ $J(w,b)$. ในการฝึกโมเดล คุณจะวัดค่า cost โดยพิจารณาจากตัวอย่างการฝึกทั้งหมด $x^{(i)},y^{(i)}$
$$J(w,b) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)})^2\tag{2}$$ 


ในบทเรียน Gradient Descent ได้อธิบายไว้ดังนี้:

$$\begin{align*} \text{repeat}&\text{ until convergence:} \; \lbrace \newline
\;  w &= w -  \alpha \frac{\partial J(w,b)}{\partial w} \tag{3}  \; \newline 
 b &= b -  \alpha \frac{\partial J(w,b)}{\partial b}  \newline \rbrace
\end{align*}$$
โดยที่:
- พารามิเตอร์ w และ b จะถูกอัปเดต พร้อมกัน
- การคำนวณอนุพันธ์ย่อย (partial derivative) ของ w และ b จะถูกคำนวณก่อนที่จะอัปเดตพารามิเตอร์ใดๆ

นิยามของ Gradient:
$$
\begin{align}
\frac{\partial J(w,b)}{\partial w}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)})x^{(i)} \tag{4}\\
  \frac{\partial J(w,b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)}) \tag{5}\\
\end{align}
$$

ความหมายของ *"simultaneously"*:

หมายถึงการคำนวณอนุพันธ์ย่อยของพารามิเตอร์ทั้งหมดก่อนที่จะอัปเดตพารามิเตอร์ใดๆ
ไม่ใช่การอัปเดตพารามิเตอร์ทีละตัว
การอัปเดตแบบพร้อมกันช่วยให้การปรับปรุงโมเดลมีประสิทธิภาพมากขึ้น

<a name="toc_40291_2.2"></a>
## ใช้อัลกอริทึม Gradient Descent
คุณจะต้องเขียนฟังก์ชัน 3 ฟังก์ชันเพื่อใช้อัลกอริทึม Gradient Descent สำหรับคุณสมบัติ (feature) เดียว
- `compute_gradient` ใช้อสมการ (4) และ (5) ด้านบน
- `compute_cost` ใช้อสมการ (2) ด้านบน (จาก Lab ก่อนหน้า)
- `gradient_descent` ช้อสมการ compute_gradient และ compute_cost

ข้อตกลงในการตั้งชื่อตัวแปร:
- ตัวแปร Python ที่เก็บค่าอนุพันธ์ย่อย จะใช้รูปแบบการตั้งชื่อดังนี้  
 $\frac{\partial J(w,b)}{\partial b}$  จะถูกตั้งชื่อเป็น dj_db
- w.r.t คือ With Respect To (เทียบกับ) เช่น อนุพันธ์ย่อยของ J(wb) เทียบกับ b


<a name="toc_40291_2.3"></a>
### compute_gradient
<a name='ex-01'></a>
`compute_gradient` ฟังก์ชันนี้ดำเนินการตามขั้นตอนที่ 4 และ 5 ด้านบน และส่งคืนค่า  
 $\frac{\partial J(w,b)}{\partial w}$,$\frac{\partial J(w,b)}{\partial b}$. คำอธิบายในโค้ดบรรยายการดำเนินการต่างๆ"

In [ ]:
def compute_gradient(x, y, w, b): 
    """
    Computes the gradient for linear regression 
    Args:
      x (ndarray (m,)): Data, m examples 
      y (ndarray (m,)): target values
      w,b (scalar)    : model parameters  
    Returns
      dj_dw (scalar): The gradient of the cost w.r.t. the parameters w
      dj_db (scalar): The gradient of the cost w.r.t. the parameter b     
     """
    
    # Number of training examples
    m = x.shape[0]    
    dj_dw = 0
    dj_db = 0
    
    for i in range(m):  
        f_wb = w * x[i] + b 
        dj_dw_i = (f_wb - y[i]) * x[i] 
        dj_db_i = f_wb - y[i] 
        dj_db += dj_db_i
        dj_dw += dj_dw_i 
    dj_dw = dj_dw / m 
    dj_db = dj_db / m 
        
    return dj_dw, dj_db

<br/>

<img align="left" src="./images/C1_W1_Lab03_lecture_slopes_v1.png"   style="width:340px;" > บทบรรยายอธิบายว่าการไล่ระดับ (gradient descent) ใช้อนุพันธ์ย่อยของต้นทุนเทียบกับพารามิเตอร์ที่จุดหนึ่งเพื่ออัปเดตพารามิเตอร์นั้น

มาใช้ฟังก์ชัน compute_gradient ของเรากันเพื่อหาและพล็อตอนุพันธ์ย่อยบางส่วนของฟังก์ชันต้นทุนของเรามากกว่าหนึ่งพารามิเตอร์ $w_0$.


In [ ]:
plt_gradients(x_train,y_train, compute_cost, compute_gradient)
plt.show()

ด้านซ้าย: แสดงกราฟของ $\frac{\partial J(w,b)}{\partial w}$ หรือความชันของเส้นโค้งค่าใช้จ่ายเทียบกับ $w$ a ที่สามจุด  ทางด้านขวาของกราฟ อนุพันธ์เป็นบวก ในขณะที่ทางด้านซ้าย อนุพันธ์เป็นลบ เนื่องจากรูปร่าง "ชาม" ของเส้นโค้ง อนุพันธ์จะนำการไล่ระดับลงไปที่ด้านล่างซึ่งมีค่าอนุพันธ์เป็นศูนย์เสมอ
 
ด้านซ้าย $b=100$ คงที่ การไล่ระดับจะใช้ทั้ง $\frac{\partial J(w,b)}{\partial w}$ และ  $\frac{\partial J(w,b)}{\partial b}$ เพื่อปรับปรัมพารามิเตอร์ "กราฟลูกศร" ทางด้านขวาแสดงวิธีการมองเห็นการไล่ระดับของทั้งสองพารามิเตอร์ ขนาดของลูกศรสะท้อนถึงขนาดของการไล่ระดับที่จุดนั้น ทิศทางและความชันของลูกศรสะท้อนถึงอัตราส่วนของ $\frac{\partial J(w,b)}{\partial w}$ และ   $\frac{\partial J(w,b)}{\partial b}$ ที่จุดนั้น

หมายเหตุ: การไล่ระดับชี้ไป ห่างจาก จุดต่ำสุด  ตรวจสอบสมการ (3) ด้านบน การไล่ระดับที่ปรับขนาดแล้วจะถูก *ลบ* ออกจากค่าปัจจุบันของ `w` หรือ `b` ทำให้พารามิเตอร์เคลื่อนที่ไปในทิศทางที่ลดค่าใช้จ่าย



<a name="toc_40291_2.5"></a>
###  Gradient Descent
ตอนนี้ที่เราคำนวณการไล่ระดับได้แล้ว  เราสามารถนำการไล่ระดับมาใช้ในฟังก์ชัน gradient_descent ตามสมการ (3) ด้านบน  รายละเอียดของการใช้งานมีการอธิบายในส่วนหมายเหตุ  ด้านล่างนี้ คุณจะใช้ฟังก์ชันนี้เพื่อหาค่าที่เหมาะสมที่สุดของ $w$ และ $b$ บนข้อมูลการฝึก

In [ ]:
def gradient_descent(x, y, w_in, b_in, alpha, num_iters, cost_function, gradient_function): 
    """
    Performs gradient descent to fit w,b. Updates w,b by taking 
    num_iters gradient steps with learning rate alpha
    
    Args:
      x (ndarray (m,))  : Data, m examples 
      y (ndarray (m,))  : target values
      w_in,b_in (scalar): initial values of model parameters  
      alpha (float):     Learning rate
      num_iters (int):   number of iterations to run gradient descent
      cost_function:     function to call to produce cost
      gradient_function: function to call to produce gradient
      
    Returns:
      w (scalar): Updated value of parameter after running gradient descent
      b (scalar): Updated value of parameter after running gradient descent
      J_history (List): History of cost values
      p_history (list): History of parameters [w,b] 
      """
    
    # An array to store cost J and w's at each iteration primarily for graphing later
    J_history = []
    p_history = []
    b = b_in
    w = w_in
    
    for i in range(num_iters):
        # Calculate the gradient and update the parameters using gradient_function
        dj_dw, dj_db = gradient_function(x, y, w , b)     

        # Update Parameters using equation (3) above
        b = b - alpha * dj_db                            
        w = w - alpha * dj_dw                            

        # Save cost J at each iteration
        if i<100000:      # prevent resource exhaustion 
            J_history.append( cost_function(x, y, w , b))
            p_history.append([w,b])
        # Print cost every at intervals 10 times or as many iterations if < 10
        if i% math.ceil(num_iters/10) == 0:
            print(f"Iteration {i:4}: Cost {J_history[-1]:0.2e} ",
                  f"dj_dw: {dj_dw: 0.3e}, dj_db: {dj_db: 0.3e}  ",
                  f"w: {w: 0.3e}, b:{b: 0.5e}")
 
    return w, b, J_history, p_history #return w and J,w history for graphing

In [ ]:
# initialize parameters
w_init = 0
b_init = 0
# some gradient descent settings
iterations = 10000
tmp_alpha = 1.0e-2
# run gradient descent
w_final, b_final, J_hist, p_hist = gradient_descent(x_train ,y_train, w_init, b_init, tmp_alpha, 
                                                    iterations, compute_cost, compute_gradient)
print(f"(w,b) found by gradient descent: ({w_final:8.4f},{b_final:8.4f})")

<img align="left" src="./images/C1_W1_Lab03_lecture_learningrate_v1.png"  style="width:340px; padding: 15px; " > 
จากข้อมูลที่ให้มา สังเกตได้ว่ากระบวนการ Gradient Descent มีคุณสมบัติหลักดังนี้:

- ค่าฟังก์ชันต้นทุน (Cost) ลดลงอย่างรวดเร็วในช่วงแรก: ตามที่อธิบายในสไลด์จากการบรรยาย 
- ค่าต้นทุนจะเริ่มสูงและลดลงอย่างรวดเร็ว อนุพันธ์ย่อย (Partial Derivatives) `dj_dw` และ `dj_db` ลดลงอย่างรวดเร็วในช่วงแรก: จากแผนภาพในสไลด์ เมื่อกระบวนการเข้าใกล้ "จุดต่ำสุดของชาม" (bottom of the bowl) อนุพันธ์ย่อยจะลดลงอย่างช้าลง เนื่องจากค่าของอนุพันธ์ย่อย ณ จุดนั้นมีค่าน้อยลง
การเรียนรู้ (Learning Rate) alpha คงที่ แต่ความก้าวหน้าลดลง: 
- แม้ว่าอัตราการเรียนรู้จะคงที่ แต่ความก้าวหน้าในการปรับน้ำหนักและค่าตัดขวางจะลดลงตามเวลา เนื่องจากอนุพันธ์ย่อยลดลง

### Cost กับการวนซ้ำของ gradient descent 
Cost กับการวนซ้ำของ gradient descent  เป็นตัววัดความก้าวหน้าที่มีประโยชน์ใน gradient descent. Cost ควรลดลงเสมอในการรันที่ประสบความสำเร็จ. การเปลี่ยนแปลงของ cost นั้นรวดเร็วมากในช่วงแรก จึงเป็นประโยชน์ในการพล็อตการลดลงเริ่มต้นในมาตราส่วนที่แตกต่างจากการลดลงสุดท้าย ในพล็อตด้านล่าง โปรดสังเกตมาตราส่วนของ cost บนแกนและขั้นตอนการวนซ้ำ


In [ ]:
# plot cost versus iteration  
fig, (ax1, ax2) = plt.subplots(1, 2, constrained_layout=True, figsize=(12,4))
ax1.plot(J_hist[:100])
ax2.plot(1000 + np.arange(len(J_hist[1000:])), J_hist[1000:])
ax1.set_title("Cost vs. iteration(start)");  ax2.set_title("Cost vs. iteration (end)")
ax1.set_ylabel('Cost')            ;  ax2.set_ylabel('Cost') 
ax1.set_xlabel('iteration step')  ;  ax2.set_xlabel('iteration step') 
plt.show()

### Predictions (การคาดการณ์)
ตอนนี้คุณได้ค้นพบค่าที่เหมาะสมที่สุดสำหรับพารามิเตอร์ $w$ และ $$ แล้ว คุณสามารถใช้โมเดลเพื่อทำนายมูลค่าบ้านโดยอิงจากพารามิเตอร์ที่เรียนรู้ของคุณได้แล้ว ตามที่คาดไว้ ค่าที่ทำนายได้นั้นใกล้เคียงกับค่าฝึกอบรมสำหรับบ้านหลังเดียวกัน นอกจากนี้ ค่าที่ไม่อยู่ในการทำนายยังสอดคล้องกับค่าที่คาดหวัง

In [ ]:
print(f"1000 sqft house prediction {w_final*1.0 + b_final:0.1f} Thousand dollars")
print(f"1200 sqft house prediction {w_final*1.2 + b_final:0.1f} Thousand dollars")
print(f"2000 sqft house prediction {w_final*2.0 + b_final:0.1f} Thousand dollars")

<a name="toc_40291_2.6"></a>
## การพล็อต
คุณสามารถแสดงความคืบหน้าของการไล่ระดับสีในระหว่างการดำเนินการโดยการพล็อตค่า cost เหนือการทำซ้ำบนพล็อตคอนทัวร์ของค่า cost (w,b).

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12, 6))
plt_contour_wgrad(x_train, y_train, p_hist, ax)


ด้านบน แผนที่ระดับความสูงแสดงค่า $cost(w,b)$ ในช่วงค่า w และ b ที่หลากหลาย ระดับค่า cost จะแสดงด้วยวงแหวน เส้นทางการไล่ระดับความชันจะทับซ้อนกันโดยใช้ลูกศรสีแดง นี่คือสิ่งที่ควรสังเกต:

- เส้นทางมีความก้าวหน้าอย่างต่อเนื่อง (ซ้ำซ้อน) ไปสู่เป้าหมาย
- ขั้นตอนเริ่มต้นมีขนาดใหญ่กว่าขั้นตอนใกล้เป้าหมายมาก

**ซูมเข้าไป**"เราจะเห็นขั้นตอนสุดท้ายของการไล่ระดับลง (gradient descent) สังเกตว่าระยะห่างระหว่างขั้นตอนจะลดลงเมื่อการไล่ระดับเข้าใกล้ศูนย์"

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12, 4))
plt_contour_wgrad(x_train, y_train, p_hist, ax, w_range=[180, 220, 0.5], b_range=[80, 120, 0.5],
            contours=[1,5,10,20],resolution=0.5)

<a name="toc_40291_2.7.1"></a>
### การเพิ่ม Learning Rate

<figure>
 <img align="left", src="./images/C1_W1_Lab03_alpha_too_big_v1.png"   style="width:340px;height:240px;" >
</figure>

ในการบรรยาย มีการพูดคุยเกี่ยวกับค่าที่เหมาะสมของอัตราการเรียนรู้ α ในสมการ (3) ค่า  $\alpha$ ที่สูงขึ้นจะทำให้การไล่ระดับลงมาบรรจบกับคำตอบได้เร็วขึ้น แต่ถ้าสูงเกินไป การไล่ระดับลงมาจะแยกออกไป

ด้านบนคุณมีตัวอย่างของคำตอบที่บรรจบกันอย่างดี

ลองเพิ่มค่า  $\alpha$ และดูว่าจะเกิดอะไรขึ้น:

In [ ]:
# initialize parameters
w_init = 0
b_init = 0
# set alpha to a large value
iterations = 10
tmp_alpha = 8.0e-1
# run gradient descent
w_final, b_final, J_hist, p_hist = gradient_descent(x_train ,y_train, w_init, b_init, tmp_alpha, 
                                                    iterations, compute_cost, compute_gradient)

ด้านบน $w$ และ $b$ กำลังเด้งกลับไปกลับมาระหว่างบวกและลบโดยมีค่าสัมบูรณ์เพิ่มขึ้นในแต่ละรอบ นอกจากนี้ ในแต่ละรอบ  $\frac{\partial J(w,b)}{\partial w}$
เปลี่ยนเครื่องหมายและค่าใช้จ่ายเพิ่มขึ้นแทนที่จะลดลง นี่เป็นสัญญาณที่ชัดเจนว่า *อัตราการเรียนรู้สูงเกินไป *และโซลูชันกำลังเบี่ยงเบน

มาลองดูภาพประกอบกัน

In [ ]:
plt_divergence(p_hist, J_hist,x_train, y_train)
plt.show()

ด้านบน กราฟทางซ้ายแสดงความคืบหน้าของ $w$ ในช่วงแรก ๆ ของการไล่ระดับลง $w$ สั่นจากบวกเป็นลบและต้นทุนเพิ่มขึ้นอย่างรวดเร็ว การไล่ระดับลงทำงานบนทั้ง $w$ และ $b$ พร้อมกัน ดังนั้นจึงต้องใช้แผนภูมิ 3 มิติทางขวาเพื่อภาพรวมที่สมบูรณ์

## ขอแสดงความยินดี!
ในห้องปฏิบัติการนี้ คุณได้:

- เจาะลึกรายละเอียดของการไล่ระดับสำหรับตัวแปรเดียว
- พัฒนาโปรแกรมเพื่อคำนวณการไล่ระดับ
- แสดงภาพว่าการไล่ระดับคืออะไร
- เสร็จสิ้นโปรแกรมการไล่ระดับ
- ใช้การไล่ระดับเพื่อหาพารามิเตอร์
- ตรวจสอบผลกระทบของการปรับขนาดอัตราการเรียนรู้